In [1]:
import pandas as pd
import xarray as xr
from IPython.display import clear_output
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
from ar6_ch6_rcmipfigs.constants import INPUT_DATA_DIR, OUTPUT_DATA_DIR, RESULTS_DIR

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs
/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_in


In [2]:
# name of output variable
name_deltaT = 'Delta T'

climatemodel = 'climatemodel'
scenario = 'scenario'
variable = 'variable'
time = 'time'
percentile = 'percentile'

### Define variables to look at:

In [3]:
# variables to include:

variables_erf_comp = [
    'ch4',
    'aerosol-total-with_bc-snow',
    'aerosol-radiation_interactions',
    'aerosol-cloud_interactions',
    'aerosol-total',
    'o3',
    'HFCs',
    # 'F-Gases|HFC',
    'bc_on_snow',
    'total_anthropogenic',
    'total',
]
variables_in_sum = [
    'aerosol-total-with_bc-snow',
    'ch4',
    # 'aerosol-radiation_interactions',
    # 'aerosol-cloud_interactions',
    #'aerosol-total',
    'o3',
    'HFCs',
    #'bc_on_snow'
]
# total ERFs for anthropogenic and total:
variables_erf_tot = ['total_anthropogenic',
                     'total']
variables_all = variables_erf_comp + variables_erf_tot
# Scenarios to plot:
scenarios_fl = ['ssp534-over', 'ssp119', 'ssp460', 'ssp585', 'ssp370',
                'ssp370-lowNTCF-aerchemmip', 'ssp126', 'ssp245', 'ssp434',
                'ssp370-lowNTCF-gidden'
                ]

In [4]:
recommendation = 'recommendation'
IRFpercentiles = [recommendation]
# {'ECS = 2K':0.526, 'ECS = 3.4K':0.884, 'ECS = 5K': 1.136 }

Year to integrate from and to:

In [5]:
first_y = 1750
last_y = 2100

**Set reference year for temperature change:**

In [6]:
ref_year = 2020

**Years to output change in**

In [7]:
years = [2040, 2100]

### Input dataset:

In [8]:
PATH_DT_INPUT = OUTPUT_DATA_DIR / 'dT_data_RCMIP_recommendation.nc'

In [9]:
PATH_DT_TAB_OUTPUT = RESULTS_DIR / 'tables_recommendation' / 'table_sens_dT_cs_recommandetion.csv'






In [10]:
print(PATH_DT_INPUT)

/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/data_out/dT_data_RCMIP_recommendation.nc


In [11]:
    
    
ds_DT = xr.open_dataset(PATH_DT_INPUT)

In [12]:
ds_DT.variable

<xarray.DataArray 'variable' (variable: 19)>
array(['co2', 'ch4', 'n2o', 'other_wmghg', 'o3', 'h2o_stratospheric',
       'contrails', 'aerosol-radiation_interactions',
       'aerosol-cloud_interactions', 'bc_on_snow', 'land_use', 'volcanic',
       'solar', 'total_anthropogenic', 'total_natural', 'total',
       'aerosol-total', 'aerosol-total-with_bc-snow', 'HFCs'], dtype=object)
Coordinates:
  * variable  (variable) object 'co2' 'ch4' ... 'HFCs'

## Table

### Setup table:

In [13]:

iterables = [list(IRFpercentiles), years]


def setup_table(scenario_n='', variables=variables_all):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    table = pd.DataFrame(columns= variables, index=_i).transpose()
    table.index.name = scenario_n
    return table

In [14]:
# Dicitonary of tables with different ESC:
scntab_dic = {}
for scn in scenarios_fl:
    # Loop over scenrarios
    tab = setup_table(scenario_n=scn, variables=variables_erf_comp)  # make table
    for var in variables_erf_comp:
        # Loop over variables
        tabvar = var.split('|')[-1]
        dtvar = name_deltaT
        for key in IRFpercentiles:
            # Loop over ESC parameters
            for year in years:
                
                da_ref_y = ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=ref_year,
                ).squeeze()
                
                da_sel_y =ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=year
                ) 
                _tab_da = da_sel_y - da_ref_y
                a = float(_tab_da.loc[var].squeeze().values)
                tab.loc[tabvar, (key, year)] = a
    scntab_dic[scn] = tab.copy()

In [15]:
from IPython.display import display

for key in scntab_dic:
    display(scntab_dic[key])

recommendation            
                                         2040        2100
ssp534-over                                              
ch4                                 0.0658166    -0.16726
aerosol-total-with_bc-snow           0.122591    0.360168
aerosol-radiation_interactions      0.0344069 -0.00232192
aerosol-cloud_interactions          0.0865606    0.403743
aerosol-total                        0.120968    0.401421
o3                                  0.0426344   -0.132793
HFCs                                0.0423203   0.0152045
bc_on_snow                         0.00162298  -0.0412535
total_anthropogenic                  0.849906    0.883605
total                                0.805609    0.825799

recommendation            
                                         2040        2100
ssp119                                                   
ch4                                -0.0328498   -0.163074
aerosol-total-with_bc-snow           0.273794    0.424517
aerosol-radiation_interactions      0.0300329   0.0390842
aerosol-cloud_interactions           0.271767    0.426961
aerosol-total                          0.3018    0.466045
o3                                 -0.0830983   -0.148473
HFCs                               0.00538032 -0.00378865
bc_on_snow                         -0.0280065  -0.0415275
total_anthropogenic                  0.397428    0.116036
total                                0.353131   0.0582305

recommendation           
                                         2040       2100
ssp460                                                  
ch4                                 0.0866686   0.179151
aerosol-total-with_bc-snow        -0.00630511   0.178273
aerosol-radiation_interactions      0.0152394 -0.0219939
aerosol-cloud_interactions         -0.0357434   0.222835
aerosol-total                      -0.0205039   0.200841
o3                                  0.0558849  0.0937053
HFCs                                0.0381366   0.040947
bc_on_snow                          0.0141988 -0.0225674
total_anthropogenic                  0.645718    2.26177
total                                 0.60142    2.20396

recommendation           
                                         2040       2100
ssp585                                                  
ch4                                 0.0618279   0.147478
aerosol-total-with_bc-snow           0.122591   0.232166
aerosol-radiation_interactions      0.0344069 -0.0063042
aerosol-cloud_interactions          0.0865606    0.26812
aerosol-total                        0.120968   0.261816
o3                                  0.0400979  0.0506565
HFCs                                0.0423128   0.223272
bc_on_snow                         0.00162298 -0.0296496
total_anthropogenic                  0.836875    4.11427
total                                0.792577    4.05646

recommendation             
                                         2040         2100
ssp370                                                    
ch4                                 0.0761809     0.276651
aerosol-total-with_bc-snow          -0.052029   -0.0266798
aerosol-radiation_interactions      0.0128722  4.43553e-05
aerosol-cloud_interactions         -0.0854859   -0.0403596
aerosol-total                      -0.0726136   -0.0403152
o3                                  0.0698454     0.236445
HFCs                                0.0264999    0.0969879
bc_on_snow                          0.0205846    0.0136354
total_anthropogenic                  0.663786      3.15208
total                                0.619488      3.09427

recommendation           
                                         2040       2100
ssp370-lowNTCF-aerchemmip                               
ch4                                 0.0761809   0.276651
aerosol-total-with_bc-snow          0.0870509   0.284753
aerosol-radiation_interactions      0.0194731  0.0168867
aerosol-cloud_interactions          0.0696048   0.295293
aerosol-total                       0.0890779    0.31218
o3                                  0.0244954   0.157736
HFCs                                0.0264999  0.0969879
bc_on_snow                        -0.00202703 -0.0274264
total_anthropogenic                   0.75756    3.37383
total                                0.713263    3.31602

recommendation            
                                         2040        2100
ssp126                                                   
ch4                                -0.0208684    -0.15534
aerosol-total-with_bc-snow            0.22836    0.434063
aerosol-radiation_interactions      0.0255283    0.047689
aerosol-cloud_interactions           0.226157    0.425775
aerosol-total                        0.251685    0.473464
o3                                 -0.0598631    -0.14714
HFCs                                0.0170608 -0.00146604
bc_on_snow                         -0.0233254  -0.0394008
total_anthropogenic                  0.505497    0.544207
total                                  0.4612    0.486401

recommendation            
                                         2040        2100
ssp245                                                   
ch4                                 0.0417628 -0.00104217
aerosol-total-with_bc-snow          0.0466276    0.281874
aerosol-radiation_interactions     0.00400846   0.0045108
aerosol-cloud_interactions          0.0450464    0.313308
aerosol-total                       0.0490549    0.317819
o3                                   0.014977  -0.0459325
HFCs                                0.0284577   0.0575687
bc_on_snow                        -0.00242724   -0.035945
total_anthropogenic                  0.584779     1.75423
total                                0.540482     1.69643

recommendation            
                                         2040        2100
ssp434                                                   
ch4                                 0.0553451    0.125203
aerosol-total-with_bc-snow          0.0756106    0.260492
aerosol-radiation_interactions      0.0302367  0.00113449
aerosol-cloud_interactions          0.0391195    0.286926
aerosol-total                       0.0693562    0.288061
o3                                  0.0256502   0.0468623
HFCs                               0.00337091 -0.00736819
bc_on_snow                         0.00625439  -0.0275684
total_anthropogenic                  0.530424     1.08861
total                                0.486127     1.03081

recommendation           
                                         2040       2100
ssp370-lowNTCF-gidden                                   
ch4                                -0.0362155  -0.122362
aerosol-total-with_bc-snow          0.0870509   0.284753
aerosol-radiation_interactions      0.0194731  0.0168867
aerosol-cloud_interactions          0.0696048   0.295293
aerosol-total                       0.0890779    0.31218
o3                                 -0.0442151 -0.0985579
HFCs                                 0.026497   0.083767
bc_on_snow                        -0.00202703 -0.0274264
total_anthropogenic                  0.566941    2.64906
total                                0.522644    2.59126

### Make table with all scenarios:

In [16]:
iterables = [list(IRFpercentiles), years]
iterables2 = [scenarios_fl, variables_erf_comp]


def setup_table2():  # scenario_n=''):
    _i = pd.MultiIndex.from_product(iterables, names=['', ''])
    _r = pd.MultiIndex.from_product(iterables2, names=['', ''])

    table = pd.DataFrame(columns=_r, index=_i).transpose()
    return table

In [17]:
tab = setup_table2()  # scenario_n=scn)

for scn in scenarios_fl:
    for var in variables_erf_comp:
        tabvar = var#.split('|')[-1]
        dtvar = name_deltaT
        for key in IRFpercentiles:
            for year in years:
                # compute difference between year and ref year
                _da_y = ds_DT[dtvar].sel(
                    percentile=key,
                    scenario=scn, 
                    year=year, 
                    variable=var
                ).squeeze()  # .squeeze()
                _da_refy = ds_DT[dtvar].sel(
                    scenario=scn, 
                    year=ref_year, 
                    variable=var,
                ).squeeze()
                # _tab_da = dic_ds[key][dtvar].sel(scenario=scn, time=slice(year,year))-  dic_ds[key][dtvar].sel(scenario=scn, time=slice(ref_year,ref_year)).squeeze()
                _tab_da = _da_y - _da_refy

                tab.loc[(scn, tabvar), (key, year)] = _tab_da.squeeze().values  # [0]

In [18]:
tab

recommendation  \
                                                                       2040   
                                                                              
ssp534-over           ch4                               0.06581660893277874   
                      aerosol-total-with_bc-snow         0.1225905250675402   
                      aerosol-radiation_interactions   0.034406920882383424   
                      aerosol-cloud_interactions        0.08656062651392965   
                      aerosol-total                     0.12096754739631299   
...                                                                     ...   
ssp370-lowNTCF-gidden o3                               -0.04421511782025506   
                      HFCs                              0.02649699726993186   
                      bc_on_snow                      -0.002027027596469076   
                      total_anthropogenic                0.5669413931677021   
                      total                              0.5226442590299389   

                                                                              
                                                                        2100  
                                                                              
ssp534-over           ch4                               -0.16725956827439928  
                      aerosol-total-with_bc-snow         0.36016801400689874  
                      aerosol-radiation_interactions  -0.0023219218621474136  
                      aerosol-cloud_interactions         0.40374341118226986  
                      aerosol-total                      0.40142148932012234  
...                                                                      ...  
ssp370-lowNTCF-gidden o3                                -0.09855791702674277  
                      HFCs                                0.0837669837537589  
                      bc_on_snow                        -0.02742636191202624  
                      total_anthropogenic                  2.649060659159763  
                      total                                 2.59125529021292  

[100 rows x 2 columns]

In [19]:
tab

recommendation  \
                                                                       2040   
                                                                              
ssp534-over           ch4                               0.06581660893277874   
                      aerosol-total-with_bc-snow         0.1225905250675402   
                      aerosol-radiation_interactions   0.034406920882383424   
                      aerosol-cloud_interactions        0.08656062651392965   
                      aerosol-total                     0.12096754739631299   
...                                                                     ...   
ssp370-lowNTCF-gidden o3                               -0.04421511782025506   
                      HFCs                              0.02649699726993186   
                      bc_on_snow                      -0.002027027596469076   
                      total_anthropogenic                0.5669413931677021   
                      total                              0.5226442590299389   

                                                                              
                                                                        2100  
                                                                              
ssp534-over           ch4                               -0.16725956827439928  
                      aerosol-total-with_bc-snow         0.36016801400689874  
                      aerosol-radiation_interactions  -0.0023219218621474136  
                      aerosol-cloud_interactions         0.40374341118226986  
                      aerosol-total                      0.40142148932012234  
...                                                                      ...  
ssp370-lowNTCF-gidden o3                                -0.09855791702674277  
                      HFCs                                0.0837669837537589  
                      bc_on_snow                        -0.02742636191202624  
                      total_anthropogenic                  2.649060659159763  
                      total                                 2.59125529021292  

[100 rows x 2 columns]

## Save output

In [20]:
tab.to_csv(PATH_DT_TAB_OUTPUT)

In [21]:
PATH_DT_TAB_OUTPUT

PosixPath('/home/sarambl/PHD/IPCC/public/AR6_CH6_RCMIPFIGSv2/ar6_ch6_rcmipfigs/results/tables_recommendation/table_sens_dT_cs_recommandetion.csv')